In [8]:

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import roc_auc_score, mean_squared_error, f1_score, accuracy_score
from sklearn.cluster import KMeans
from lightgbm import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import roc_auc_score, mean_squared_error, f1_score, accuracy_score, recall_score
from sklearn.cluster import KMeans
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from imblearn.metrics import geometric_mean_score
from sklearn.ensemble import RandomForestClassifier

In [9]:
data = pd.read_csv("dataset/creditcard.csv")
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [10]:
data['Class'].value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

In [11]:
print("Persentage of Y: \n")
data['Class'].value_counts()/len(data['Class'])*100

Persentage of Y: 



Class
0    99.827251
1     0.172749
Name: count, dtype: float64

In [12]:
print("Data imbalance ratio: " ,492/284315 )

Data imbalance ratio:  0.0017304750013189597


In [13]:
y = data['Class']
X=data.drop('Class', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [14]:

# Utility functions
def stats_mv(X):
    # Implement missing value statistics
    return np.sum(np.isnan(X), axis=0)

def corr_ana(X):
    # Implement correlation analysis
    return np.corrcoef(X, rowvar=False)

def cpd_group(Fm, Fc):
    # Implement compound grouping
    Gm = np.where(Fm > 0)[0]
    Gc = np.where(abs(Fc) > 0.8)  # Example threshold for high correlation
    return Gm, Gc

def update_feature_count(X):
    # Implement feature count updating
    return X.shape[1]

def KMeans_plus_plus(X, n_clusters):
    # Implement K-Means++ clustering
    kmeans = KMeans(n_clusters=n_clusters, init='k-means++', n_init=10).fit(X)
    return kmeans.cluster_centers_, kmeans.labels_

def construct_spiral(Ccen, X_fraud):
    # Extract center coordinates
    x_center, y_center = Ccen

    # Extract coordinates of fraud data
    x_fraud, y_fraud = X_fraud

    # Calculate distances from center to fraud data points
    distances = np.sqrt((x_fraud - x_center) ** 2 + (y_fraud - y_center) ** 2)

    # Generate angles for spiral curve based on distances
    angles = distances * 2 * np.pi

    # Generate spiral curve coordinates
    x_spiral = x_center + distances * np.cos(angles)
    y_spiral = y_center + distances * np.sin(angles)

    return x_spiral, y_spiral

def map_to_linear_space(Artisam):
    linear_space_samples = []
    for sample in Artisam:
        if len(sample) > 0:
            linear_space_samples.append(np.concatenate(sample, axis=1))

    if len(linear_space_samples) > 0:
        return np.concatenate(linear_space_samples)
    else:
        return np.array([])


In [15]:

# CGEA (Compound Grouping Elimination Algorithm)
# Step 1: Group features by missing values
Fm = stats_mv(X)

# Step 2: Group features by correlation
Fc = corr_ana(X)
Gm, Gc = cpd_group(Fm, Fc)

# Step 3: Eliminate redundant features
Fres = []
N_f_amt, N_f_max = 0, 0
for i in range(len(Gm)):
    for j in range(len(Gc)):
        N_f_amt = update_feature_count(X_train)
        if N_f_amt >= N_f_max:
            N_f_max = N_f_amt
            Fres.append(Gc[j])


In [16]:
# MSEFBoost (Multifactor Synchronous Embedding Feature Boosting)
from sklearn.metrics import roc_auc_score, mean_squared_error
from lightgbm import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor

# Train decision tree regression model
dtr = XGBClassifier()
dtr.fit(X_train, y_train)

# Predict using the trained model
y_pred = dtr.predict(X_train)

# Calculate AUC and MSE
VAUC = roc_auc_score(y_train, y_pred)
VMSE = mean_squared_error(y_train, y_pred)
VAMsub = VAUC - VMSE

# Train LightGBM classifier
lgbm = LGBMRegressor()
lgbm.fit(X_train, y_train)

# Get feature importances
feature_importances = lgbm.feature_importances_

# Calculate metamorphic factor
meFac = feature_importances / sum(feature_importances) + (VAUC - VMSE) ** -1


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011787 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7650
[LightGBM] [Info] Number of data points in the train set: 213605, number of used features: 30
[LightGBM] [Info] Start training from score 0.001774


In [17]:
from multiprocessing import Pool, cpu_count
import numpy as np

# Function to construct spiral model for a given cluster
def construct_spiral_model(args):
    cluster_index, cluster_data, cluster_label = args
    Min_prop = sum(cluster_label == 1) / sum(y_train == 1)
    NC_count = int(Min_prop * Nsample)
    if Min_prop == 0:
        return np.array([])
    spiral_samples = []
    for _ in range(NC_count):
        Z = construct_spiral(Ccen[cluster_index], [cluster_data[:, 0], cluster_data[:, 1]])
        X_new = np.sin(Z) * Z
        Y_new = np.cos(Z) * Z
        spiral_samples.append([X_new, Y_new, Z])
    return np.array(spiral_samples)

# SOBT (Spiral Oversampling Balancing Technique)
# Step 1: Clustering
n_clusters = 2
Ccen, Ctag = KMeans_plus_plus(X_train, n_clusters)

# Step 2: Construct 3D spiral models in parallel
Nsample = sum(y_train == 1) - sum(y_train == 0)
num_workers = cpu_count()
pool = Pool(processes=num_workers)
args_list = [(i, X_train[Ctag == i], y_train[Ctag == i]) for i in range(len(Ccen))]
spiral_samples_list = pool.map(construct_spiral_model, args_list)
pool.close()
pool.join()

# Flatten the list of lists
Artisam = np.concatenate(spiral_samples_list, axis=0)

# Step 3: Generate artificial samples
Samplere = map_to_linear_space(Artisam)

# Check if Samplere is one-dimensional and reshape if necessary
if Samplere.ndim == 1:
    Samplere = Samplere.reshape(-1, 1)

# Check the shape of Samplere and reshape if necessary
if Samplere.shape[1] != X_train.shape[1]:
    Samplere = Samplere.reshape(-1, X_train.shape[1])

# Step 4: Combine artificial samples with original data for oversampling
X_SOBT_Oversampled = np.concatenate((X_train, Samplere), axis=0)
Y_SOBT_Oversampled = np.concatenate((y_train, np.ones(len(Samplere))))

# Feature boosting on oversampled data
X_SOBT_boosted = X_SOBT_Oversampled * meFac
Y_SOBT_boosted = Y_SOBT_Oversampled

In [ ]:
from sklearn.model_selection import train_test_split

# Split the oversampled data into train and test sets
X_train_SOBT_oversampled, X_test_SOBT_oversampled, y_train_SOBT_oversampled, y_test_SOBT_oversampled = train_test_split(X_SOBT_Oversampled, Y_SOBT_Oversampled, test_size=0.25, random_state=42)

# Split the boosted data into train and test sets
X_train_SOBT_boosted, X_test_SOBT_boosted, y_train_SOBT_boosted, y_test_SOBT_boosted = train_test_split(X_SOBT_boosted, Y_SOBT_boosted, test_size=0.25, random_state=42)


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier



In [ ]:
def evaluate_model(model, X_train, y_train, X_test, y_test):
    # Train the model
    model.fit(X_train, y_train)

    # Predictions on the test set
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)

    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn + fp)
    g_mean = (recall * specificity) ** 0.5

    return accuracy, precision, recall, f1, auc, g_mean



In [ ]:
# Initialize models
lgbm_model_oversampled = LGBMClassifier()
rf_model_oversampled = RandomForestClassifier()

# Evaluate LightGBM model for oversampled data
lgbm_metrics_oversampled = evaluate_model(lgbm_model_oversampled, X_train_SOBT_oversampled, y_train_SOBT_oversampled, X_test_SOBT_oversampled, y_test_SOBT_oversampled)

# Evaluate Random Forest model for oversampled data
rf_metrics_oversampled = evaluate_model(rf_model_oversampled, X_train_SOBT_oversampled, y_train_SOBT_oversampled, X_test_SOBT_oversampled, y_test_SOBT_oversampled)


[LightGBM] [Info] Number of positive: 18, number of negative: 4466
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001520 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7650
[LightGBM] [Info] Number of data points in the train set: 4484, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.004014 -> initscore=-5.513877
[LightGBM] [Info] Start training from score -5.513877
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

In [ ]:
# Initialize models
lgbm_model_boosted = LGBMClassifier()
rf_model_boosted = RandomForestClassifier()

# Evaluate LightGBM model for boosted data
lgbm_metrics_boosted = evaluate_model(lgbm_model_boosted, X_train_SOBT_boosted, y_train_SOBT_boosted, X_test_SOBT_boosted, y_test_SOBT_boosted)

# Evaluate Random Forest model for boosted data
rf_metrics_boosted = evaluate_model(rf_model_boosted, X_train_SOBT_boosted, y_train_SOBT_boosted, X_test_SOBT_boosted, y_test_SOBT_boosted)


[LightGBM] [Info] Number of positive: 18, number of negative: 4466
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001468 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7650
[LightGBM] [Info] Number of data points in the train set: 4484, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.004014 -> initscore=-5.513877
[LightGBM] [Info] Start training from score -5.513877
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

In [ ]:
# Print evaluation metrics for oversampled data
print("Metrics for oversampled data:")
print("LightGBM: Accuracy: {:.2f}, Precision: {:.2f}, Recall: {:.2f}, F1-score: {:.2f}, AUC: {:.2f}, G-mean: {:.2f}".format(*lgbm_metrics_oversampled))
print("Random Forest: Accuracy: {:.2f}, Precision: {:.2f}, Recall: {:.2f}, F1-score: {:.2f}, AUC: {:.2f}, G-mean: {:.2f}".format(*rf_metrics_oversampled))

# Print evaluation metrics for boosted data
print("Metrics for boosted data:")
print("LightGBM: Accuracy: {:.2f}, Precision: {:.2f}, Recall: {:.2f}, F1-score: {:.2f}, AUC: {:.2f}, G-mean: {:.2f}".format(*lgbm_metrics_boosted))
print("Random Forest: Accuracy: {:.2f}, Precision: {:.2f}, Recall: {:.2f}, F1-score: {:.2f}, AUC: {:.2f}, G-mean: {:.2f}".format(*rf_metrics_boosted))


Metrics for oversampled data:
LightGBM: Accuracy: 0.98, Precision: 0.11, Recall: 0.60, F1-score: 0.19, AUC: 0.73, G-mean: 0.77
Random Forest: Accuracy: 1.00, Precision: 1.00, Recall: 0.80, F1-score: 0.89, AUC: 1.00, G-mean: 0.89
Metrics for boosted data:
LightGBM: Accuracy: 0.98, Precision: 0.11, Recall: 0.60, F1-score: 0.19, AUC: 0.73, G-mean: 0.77
Random Forest: Accuracy: 1.00, Precision: 1.00, Recall: 0.80, F1-score: 0.89, AUC: 1.00, G-mean: 0.89
